In [1]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms



In [ ]:
#TODO
#add more parameters
#add a confusion matrix
#mess around with hyperparameters
#related research - increasing kernel size 
#grid search/ray tune 

#data augementation 
#mnist 
#noise 
#-30 to 30 degree rotations

In [2]:
#preprocessing data
transform = transforms.Compose([
transforms.ToTensor(), #turns JPG to Tensor Object
transforms.Grayscale(num_output_channels=1), #turns the 3 layers RGB to 1 layer black and white,
transforms.Normalize(mean=[0.5,], std=[0.5,]) #some nornmalization of the grayscale,
])


In [3]:
#create a dataset object of the file system
data = torchvision.datasets.ImageFolder(root='./archive/data/extracted_images/', transform=transform)

In [4]:
#making the splits for the dataset
#taking 75% as train and 25% as hold out for test
trainsize = int(len(data)*0.75)
testsize = len(data)-trainsize

In [5]:
#splitting randomly our dataset into our chunks
train_set, test_set = torch.utils.data.random_split(data, (trainsize,testsize))

In [6]:
#creating the dataloader object (basically an iterator through our dataset that batches our files together) 
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True,  num_workers=1)
test_data_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True,  num_workers=1)

In [7]:
#defining our ConvNet
class Net(nn.Module):   
  def __init__(self):
      super(Net, self).__init__()

      self.cnn_layers = nn.Sequential(
          # Defining a 2D convolution layer
          nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(4),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          # Defining another 2D convolution layer
          nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(4),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
      )

      self.linear_layers = nn.Sequential(
          nn.Linear(484, 82)
      )

  # Defining the forward pass    
  def forward(self, x):
      x = self.cnn_layers(x)
      x = x.view(x.size(0), -1)
      x = self.linear_layers(x)
      return x

In [8]:
model = Net()
# defining the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    #put our loss and model on GPU
    #this makes it faster as GPUs have lots of multiprocessing capabilities and deal well with Matrix Multiplication and vectors 
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=484, out_features=82, bias=True)
  )
)


In [13]:
#train for 10 epochs
from tqdm import tqdm 
for i in range(10):
    print("Epoch ", i+1)
    #creating loss per epoch and correct in training set 
    running_loss = 0
    correct = 0
    #loop through our dataloader
    for images, labels in tqdm(train_data_loader):
        #put data on GPU
        if torch.cuda.is_available():
          images = images.cuda()
          labels = labels.cuda()

        # Training pass
        optimizer.zero_grad()
        
        #forward pass through our model
        output = model(images)
        #get the predicitions for that batch
        _, predicted = torch.max(output.data, 1)
        #calculate the loss 
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        #keep track of our loss and sum
        running_loss += loss.item()
        correct += (predicted == labels).sum().item()
    accuracy = 100 * (correct / (len(train_data_loader)*32))
    print(accuracy)
    print("Epoch {} - Training loss: {}".format(i+1, running_loss/len(train_data_loader)))

Epoch  1


100%|██████████| 8812/8812 [02:21<00:00, 62.39it/s]


90.28207274171585
Epoch 1 - Training loss: 0.33818531701936505
Epoch  2


100%|██████████| 8812/8812 [02:09<00:00, 68.07it/s]


91.32645823876531
Epoch 2 - Training loss: 0.298522842145084
Epoch  3


...

92.97938890149796
Epoch 8 - Training loss: 0.23270885547733694
Epoch  9


100%|██████████| 8812/8812 [02:06<00:00, 69.87it/s]


93.11521221062188
Epoch 9 - Training loss: 0.22743987918363634
Epoch  10


100%|██████████| 8812/8812 [02:06<00:00, 69.89it/s]

93.32125226963231
Epoch 10 - Training loss: 0.2232431317265287


In [19]:
#test loop
#pretty much the same thing as the dev but we do not back prop the error
y_pred = []
y_true = []
correct_count, all_count = 0, 0
with torch.no_grad():
  for inputs, labels in test_data_loader:
    inputs, labels = inputs.cuda(), labels.cuda()
    output = model(inputs) # Feed Network

    output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
    y_pred.extend(output) # Save Prediction
    
    labels = labels.data.cpu().numpy()
    y_true.extend(labels) # Save Tru